# Equal Weight S&P 500 Index Fund 
The S&P 500 is the world's most popular stock market index. It is a free-float weighted index. It is a stock market index whose components are weighted according to the total market value of their outstanding shares. But, in this project I have used equal weight S&P 500 index fund, wherein every company is assumed to be equally weighted, thus equally allocated.

The goal of this project is to create a python script that will accept the value of your portfolio size and tell you how many shares of each S&P 500 constituents you should purchase.

First import all the necessary libraries

In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')#reading the csv file containing the stocks name mentioned on S&P 500

In [3]:
from secrets import IEX_CLOUD_API_TOKEN

Loading the stock data contained in the API and appending it to the dataframe. First I have shown an example taking apple 'AAPL' stock and appending it to the dataframe. Then I have used loops to do the same to include every stock. 

In [4]:
symbol='AAPL'
api_url=f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data =  requests.get(api_url).json()

In [5]:
price = data['latestPrice']
market_cap = data['marketCap']

In [6]:
my_columns = ['Ticker', 'Stock Price', 'Market Capitalization', 'Number of shares to buy']
df = pd.DataFrame(columns=my_columns)

In [7]:
df.append(pd.Series([symbol, price, market_cap, 'N/A'], index=my_columns), ignore_index=True)

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,AAPL,144.8,2378293749055,N/A


# This will take a while.

In [8]:
for stock in stocks['Ticker']:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{stock}/quote/?token={IEX_CLOUD_API_TOKEN}'
    data =  requests.get(api_url).json()
    df = df.append(pd.Series([stock, data['latestPrice'], data['marketCap'], 'N/A'], index=my_columns), ignore_index=True)

In [9]:
df

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,157.52,47580091982,N/A
1,AAL,21.68,14515984369,N/A
2,AAP,209.00,13322006598,N/A
3,AAPL,147.55,2427767252722,N/A
4,ABBV,110.76,202100591288,N/A
...,...,...,...,...
500,YUM,124.76,37193998884,N/A
501,ZBH,156.19,32507726895,N/A
502,ZBRA,528.13,27377952626,N/A
503,ZION,64.69,10405001204,N/A


Since, http request can be extremely slow if the data is this large and we are looping over it. To avoid computational complexity, we will perform the above tasks in batches in size. This will produce the dataframe within seconds.

In [39]:
def sublist(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i+n]

In [40]:
subgroups = list(sublist(stocks['Ticker'], 100))

In [41]:
subgroups

[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 195    FTNT
 196     FTV
 197      GD
 198      GE
 199    GILD
 Name: Ticker, Length: 100, dtype: object,
 200     GIS
 201      GL
 202     GLW
 203      GM
 204    GOOG
        ... 
 295     MAA
 296     MAR
 297     MAS
 298     MCD
 299    MCHP
 Name: Ticker, Length: 100, dtype: object,
 300     MCK
 301     MCO
 302    MDLZ
 303     MDT
 304     MET
        ... 
 395     RHI
 396     RJF
 397      RL
 398     RMD
 399     ROK
 Name: Ticker, Length: 100, dtype: object,
 400     ROL
 401     ROP
 402    ROST
 403     RSG
 404     RTX
        ... 
 495    XLNX
 496     XOM
 497    XRAY
 498     XRX
 499     XYL
 Name: Ticker, Length: 100, dtype: object,
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name

In [42]:
strings = []
for i in range(0, len(subgroups)):
    strings.append(','.join(subgroups[i]))

df = pd.DataFrame(columns = my_columns)

for symbol_string in strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        #print(symbol)
        df = df.append(pd.Series([symbol,
                                 data[symbol]['quote']['latestPrice'],
                                 data[symbol]['quote']['marketCap'],
                                 'N/A'], index=my_columns), ignore_index=True)

In [43]:
df

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,158.30,49071960512,N/A
1,AAL,21.81,14420212823,N/A
2,AAP,217.00,13670430459,N/A
3,AAPL,149.18,2401096214399,N/A
4,ABBV,111.71,193959422890,N/A
...,...,...,...,...
500,YUM,127.21,36765188772,N/A
501,ZBH,152.66,32263381521,N/A
502,ZBRA,517.08,28329839133,N/A
503,ZION,63.79,10512149436,N/A


Now, we will assign number of shares one should buy based on the portfolio size. I have taken $1000000 as my portfolio size.

In [44]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not valid! \n Please enter the value in digits:")
    portfolio_size = input("Enter the value of your portfolio:")
    val = float(portfolio_size)

Enter the value of your portfolio:1000000


In [46]:
pos_size = val/len(df.index)

In [47]:
df['Number of shares to buy']= (pos_size/df['Stock Price'])

In [50]:
df['Number of shares to buy'] = df['Number of shares to buy'].apply(np.floor)

In [51]:
df

,Ticker,Stock Price,Market Capitalization,Number of shares to buy
0,A,158.30,49071960512,12.0
1,AAL,21.81,14420212823,90.0
2,AAP,217.00,13670430459,9.0
3,AAPL,149.18,2401096214399,13.0
4,ABBV,111.71,193959422890,17.0
...,...,...,...,...
500,YUM,127.21,36765188772,15.0
501,ZBH,152.66,32263381521,12.0
502,ZBRA,517.08,28329839133,3.0
503,ZION,63.79,10512149436,31.0


Adding the dataframe to the excel file so that one can explain everything to non-technical or even the management of the company very easily.

In [74]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='Recommended Trades', index = False)

# The below formatting is optional.

In [ ]:

#column_formats = { 
#'A': ['Ticker', 'string_format'],
#                   'B': ['Price', 'dollar_format'],
#                  'C': ['Market Capitalization', 'dollar_format'],
#                 'D': ['Number of Shares to Buy', 'integer_format']
#                }

#for column in column_formats.keys():
#   writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
#  writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], 'string_format')

In [75]:
writer.save()#saving the excel file to the computer